In [246]:
import argparse

parser = argparse.ArgumentParser()

# parser.add_argument('--foo', action='store_const', const=42)
parser.add_argument("--foo")

parser.parse_args(['--foo', "33"])

Namespace(foo='33')

In [250]:
import os

os.getcwd()

'/home/ryanwtsai/repos/ml_ai_portfolio/titanic/notebooks'

In [18]:
import sys
from pathlib import Path

proj_root = Path().resolve().parents[0]

sys.path.append(str(proj_root))
print(*sys.path, sep="\n")

/usr/lib/python312.zip
/usr/lib/python3.12
/usr/lib/python3.12/lib-dynload

/home/ryanwtsai/repos/ml_ai_portfolio/notes/example_code/venv/lib/python3.12/site-packages
/home/ryanwtsai/repos/ml_ai_portfolio/titanic
/home/ryanwtsai/repos/ml_ai_portfolio/titanic


In [ ]:
from src.utils import ROOT, DATA_DIR, EXPERIMENTS_DIR, OUTPUT_DIR, DATA_PATHS

sys.path.append(str(ROOT.resolve()))
sys.path.append(str(DATA_DIR.resolve()))
sys.path.append(str(EXPERIMENTS_DIR.resolve()))
sys.path.append(str(OUTPUT_DIR.resolve()))

['/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/home/ryanwtsai/repos/ml_ai_portfolio/notes/example_code/venv/lib/python3.12/site-packages',
 '/home/ryanwtsai/repos/ml_ai_portfolio/titanic',
 '/home/ryanwtsai/repos/ml_ai_portfolio/titanic',
 '/home/ryanwtsai/repos/ml_ai_portfolio/titanic',
 '/home/ryanwtsai/repos/ml_ai_portfolio/titanic/dataset',
 '/home/ryanwtsai/repos/ml_ai_portfolio/titanic/experiments',
 '/home/ryanwtsai/repos/ml_ai_portfolio/titanic/outputs']

In [235]:
from importlib import reload
import src
import experiment_config

reload(src.utils)
reload(src.custom_transformers)
reload(src.custom_model_selection)
reload(experiment_config)

<module 'experiment_config' from '/home/ryanwtsai/repos/ml_ai_portfolio/titanic/experiments/experiment_config.py'>

In [172]:
from src.custom_transformers import DynamicDataPrepPipeline
from sklearn.linear_model import LogisticRegression

age_imputer = LogisticRegression()

data_prep = DynamicDataPrepPipeline(age_imputer_model=age_imputer, fam_kwargs={"max_famsize": 4})

In [173]:
data_prep.fam_kwargs["key"] = "value"

In [174]:
from sklearn.base import clone

data_prep_clone = clone(data_prep)
data_prep.age_imputer_model is data_prep_clone.age_imputer_model # returns False

False

In [175]:
data_prep_clone is data_prep
data_prep_clone == data_prep

False

In [176]:
data_prep.fam_kwargs is data_prep_clone.fam_kwargs

False

In [177]:
data_prep_clone.fam_kwargs

{'max_famsize': 4, 'key': 'value'}

In [188]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer, MinMaxScaler, OneHotEncoder

numeric_transformations = {
    "Fare": Pipeline([("bin", KBinsDiscretizer(n_bins=4, encode="ordinal")), ("scale", MinMaxScaler())]),
    "Pclass": MinMaxScaler(),
    "FamilySize": OneHotEncoder(handle_unknown="ignore"),
    "SibSp": "passthrough"
}

In [189]:
from src.custom_transformers import DynamicDataPrepPipeline

nt_copy = DynamicDataPrepPipeline.clone_estimator_dict(numeric_transformations)

In [194]:
nt_copy["Fare"] is numeric_transformations["Fare"]
nt_copy["Pclass"] is numeric_transformations["Pclass"]
nt_copy["FamilySize"] is numeric_transformations["FamilySize"]
nt_copy["SibSp"] is numeric_transformations["SibSp"]

True

In [213]:
from experiment_config import config

config

{'data_prep': {'type': 'DynamicDataPrepPipeline',
  'params': {'ordinal_columns': ['Sex'],
   'numeric_columns': ['Age', 'Pclass', 'Fare'],
   'extract_title': True,
   'extract_fam': True,
   'extract_deck': True,
   'extract_sexpclassage': True}},
 'model': {'type': 'RandomForestClassifier',
  'params': {'max_depth': 20, 'random_state': 0}},
 'splitter': {'type': 'StratifiedKFold',
  'params': {'n_splits': 10, 'shuffle': True, 'random_state': 0}},
 'numeric_transformations': {'Fare': {'type': 'Pipeline',
   'steps': [{'name': 'bin',
     'transformer': {'type': 'KBinsDiscretizer',
      'params': {'n_bins': 4, 'encode': 'ordinal'}}},
    {'name': 'scale', 'transformer': {'type': 'MinMaxScaler'}}]},
  'Pclass': {'type': 'MinMaxScaler'}},
 'age_imputer_model': {'type': 'RandomForestRegressor',
  'params': {'max_depth': 20}}}

In [215]:
from src.utils import build_estimator, build_estimators

build_estimators(config)

DynamicDataPrepPipeline(age_imputer_model=RandomForestRegressor(max_depth=20),
                        extract_deck=True, extract_fam=True,
                        extract_sexpclassage=True, extract_title=True,
                        numeric_columns=['Age', 'Pclass', 'Fare'],
                        numeric_transformations={'Fare': Pipeline(steps=[('bin',
                                                                          KBinsDiscretizer(encode='ordinal',
                                                                                           n_bins=4)),
                                                                         ('scale',
                                                                          MinMaxScaler())]),
                                                 'Pclass': MinMaxScaler()},
                        ordinal_columns=['Sex'])


{'ml_pipe': Pipeline(steps=[('data_prep',
                  DynamicDataPrepPipeline(age_imputer_model=RandomForestRegressor(max_depth=20),
                                          extract_deck=True, extract_fam=True,
                                          extract_sexpclassage=True,
                                          extract_title=True,
                                          numeric_columns=['Age', 'Pclass',
                                                           'Fare'],
                                          numeric_transformations={'Fare': Pipeline(steps=[('bin',
                                                                                            KBinsDiscretizer(encode='ordinal',
                                                                                                             n_bins=4)),
                                                                                           ('scale',
                                                           

In [242]:
from src.utils import find_project_root, load_titanic_data

print(find_project_root())
X, y, X_test = load_titanic_data(load_X_y=False)
print(X.columns)
print(y)
print(X_test.columns if X_test is not None else X_test)
# print(data["train"].info())
# print(data["test"].info())

/home/ryanwtsai/repos/ml_ai_portfolio/titanic
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
None
None


In [234]:
data["test"].columns
# data["train_y"]

KeyError: 'test'

In [64]:
from sklearn.pipeline import Pipeline
from src.custom_transformers import FamilySizeExtractor, TitleExtractor, DeckExtractor, SexPclassAgeExtractor

def feature_extraction():
    pipe = Pipeline([
        ("fam", FamilySizeExtractor()),
        ("title", TitleExtractor()),
        ("deck", DeckExtractor()),
        ("sexpclassage", SexPclassAgeExtractor()),
    ])

    data = load_titanic_data()
    data["train"] = pipe.transform(data["train"])
    data["test"] = pipe.transform(data["test"])

    root_dir = find_project_root()
    data["train"].to_csv(Path(root_dir) / data_paths["train_extracted"], index=False)
    data["test"].to_csv(Path(root_dir) / data_paths["test_extracted"], index=False)

feature_extraction()

In [69]:
from src.custom_transformers import feature_extraction

feature_extraction()

In [80]:
from src.custom_model_selection import make_stratified_k_fold_with_custom_strata, custom_cross_validate
from src.utils import load_titanic_data

data = load_titanic_data(extracted=True)
cv_iter = make_stratified_k_fold_with_custom_strata(data["train"], ["Title", "Survived"])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from src.custom_transformers import DynamicDataPrepPipeline

ml_pipe = Pipeline([
    ("data_prep", DynamicDataPrepPipeline(extract_title=True)),
    ("model", RandomForestClassifier(n_estimators=20, random_state=0))
])

df_results, metrics = custom_cross_validate(ml_pipe, data["train"].drop(columns="Survived"), data["train"]["Survived"], cv=cv_iter)
metrics

/home/ryanwtsai/repos/ml_ai_portfolio/notes/example_code/venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


In [104]:
import yaml

config_path = proj_root / Path("config")
# yaml.safe_load(next(config_path.glob("*.yaml")))
with open(next(config_path.glob("*.yaml")), 'r') as file:
    data = yaml.safe_load(file)

data

{'experiment_name': 'rf_gridsearch_001',
 'task_type': 'grid_search',
 'pipeline_kwargs': {'use_title': True, 'use_fare': True, 'model': 'rf'},
 'param_grid': {'clf__n_estimators': [100, 200], 'clf__max_depth': [3, 5, 10]},
 'cv': {'n_splits': 5, 'shuffle': True, 'random_state': 42},
 'scoring': 'accuracy'}

In [ ]:
from src.custom_transformers import DynamicDataPrepPipeline
from sklearn.ensemble import RandomForestRegressor

data_prep_pipe = DynamicDataPrepPipeline(
    ordinal_columns={"Sex"},
    numeric_columns={"Age", "Pclass", "Fare"},
    extract_title=True,
    extract_fam=True,
    extract_deck=True,
    extract_sexpclassage=True,
    age_imputer_model=RandomForestRegressor(max_depth=10, random_state=0),
    impute_age_kwargs={"add_indicator": True},
)

params = data_prep_pipe.get_params(deep=False)
params
# model = RandomForestClassifier(max_depth=20, random_state=0)

In [ ]:
DynamicDataPrepPipeline(**params)